<a href="https://colab.research.google.com/github/kishormaity/vaccine-prediction-submission/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 432.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

file_path = '/content/prepared_data.csv'
# Load the dataset into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe to inspect the data
df.head()

,input,response
0,What is the process to file a claim?,"To file a claim, please visit our website and ..."
1,How can I check the status of my claim?,You can check the status of your claim by logg...
2,What documents are required to file a claim?,The required documents to file a claim include...
3,How long does it take to process a claim?,The processing time for a claim varies dependi...
4,Can I file a claim online?,"Yes, you can file a claim online by visiting o..."


In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Example of cleaning and tokenization (after downloading 'punkt')
import re
from nltk.tokenize import word_tokenize

# Function for text cleaning
def clean_text(text):
    text = ' '.join(text.split())
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Apply cleaning and tokenization to input and response columns
df['input'] = df['input'].apply(clean_text)
df['response'] = df['response'].apply(clean_text)

df['input'] = df['input'].apply(word_tokenize)
df['response'] = df['response'].apply(word_tokenize)

# Display the preprocessed dataframe
print(df.head())

                                               input  \
0       [what, is, the, process, to, file, a, claim]   
1   [how, can, i, check, the, status, of, my, claim]   
2  [what, documents, are, required, to, file, a, ...   
3  [how, long, does, it, take, to, process, a, cl...   
4                   [can, i, file, a, claim, online]   

                                            response  
0  [to, file, a, claim, please, visit, our, websi...  
1  [you, can, check, the, status, of, your, claim...  
2  [the, required, documents, to, file, a, claim,...  
3  [the, processing, time, for, a, claim, varies,...  
4  [yes, you, can, file, a, claim, online, by, vi...  


In [ ]:
!pip install transformers

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast

In [ ]:
# Combine input and response columns into a single text field for training
df['text'] = df['input'].astype(str) + " " + df['response'].astype(str)

# Save the combined text into a new CSV file for simplicity
combined_text_path = '/content/combined_text.csv'
df[['text']].to_csv(combined_text_path, index=False, header=False)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(tokenizer, file_path, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size  # Adjust block size as needed
    )

train_dataset = load_dataset(tokenizer, combined_text_path)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # mlm (masked language modeling) should be False for GPT-2
)

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 785.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 31.5 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", li

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',  # Output directory
    overwrite_output_dir=True,  # Overwrite the content of the output directory
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=8,  # Batch size for training
    save_steps=10_000,  # Save checkpoint every 10,000 steps
    save_total_limit=2,  # Only keep the last 2 checkpoints
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=200,  # Log every 200 steps
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [ ]:
trainer.train()

In [ ]:
# Save the trained model and tokenizer
model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_model')

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/merges.txt',
 './trained_model/added_tokens.json')

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the trained model and tokenizer
model_path = './trained_model'
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

In [ ]:
def generate_response(input_text):
    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate the output (response) from the model
    output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)

    # Decode the output to get the response text
    response_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return response_text

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import logging
from datetime import datetime
import re

# Load the trained model and tokenizer
model_path = './trained_model'
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Manually set the pad token
tokenizer.pad_token = tokenizer.eos_token

# Setup logging to record conversations
logging.basicConfig(filename='chatbot_conversations.log', level=logging.INFO)

def log_conversation(user_input, bot_response):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    logging.info(f"{timestamp} - User: {user_input}")
    logging.info(f"{timestamp} - Bot: {bot_response}")

def clean_text(text):
    # Basic text cleaning to remove unwanted characters
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n+', ' ', text)
    text = text.strip()
    return text

def generate_response(input_text, top_k=50, temperature=0.7, length_penalty=1.0):
    # Define a prompt to encourage the model to generate responses
    prompt = f"User: {input_text}\nBot:"

    # Encode the prompt text
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    attention_mask = input_ids.ne(tokenizer.pad_token_id).long()

    # Generate a single output (response) from the model using top-k sampling
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=150,
        num_return_sequences=1,  # Generate only one response
        no_repeat_ngram_size=2,
        early_stopping=True,
        do_sample=True,
        top_k=top_k,
        temperature=temperature,
        length_penalty=length_penalty,
        pad_token_id=tokenizer.eos_token_id  # Set the pad token ID to EOS token ID
    )[0]

    # Decode the output to get the response text
    response = clean_text(tokenizer.decode(output, skip_special_tokens=True).replace(prompt, "").strip())

    return response

def chat_with_bot():
    print("Bot: Hi! How can I assist you today? (Type 'exit' to end the chat)")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == 'exit':
            print("Bot: Goodbye!")
            break
        if not user_input:
            print("Bot: Please enter a valid question.")
            continue

        response = generate_response(user_input, top_k=50, temperature=0.7, length_penalty=1.0)  # Generate 1 response
        log_conversation(user_input, response)

        # Display the bot response
        print(f"Bot: {response}")

# Start the chat interface
chat_with_bot()

Bot: Hi! How can I assist you today? (Type 'exit' to end the chat)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Bot: Hello Please enter your name, and the full name of the person you want to contact. "Your name and any other information you may have regarding this customer service issue should be sent to: customerservice@gmail.com You can also use our automated email service.
Bot: What are you doing there? Anonymous: I'm going to go on a vacation. Is that what this is for? Are you planning on staying longer than I originally planned? I don't know. I guess I'll need to wait until tomorrow. _______________________________________________ "Anonymous": You know, it's cool to have a social life. What do you do when you don´t have enough money to buy things? You can't just go to your local grocery store and buy a bunch of stuff. You need something to store it. Or you can go find something cheaper and find a room in your house to sleep in. Maybe you should go out to eat a meal for someone.
You: exit
Bot: Goodbye!
